In [ ]:
using CSV
using Dates, DataFrames, Distributions, DSP
using FFTW
#using Gtk
using LaTeXStrings
using NativeFileDialog
using Plots
using Printf
using Statistics #, StatsPlots
using Tk

include("./read_BVA_processing_tools.jl")
include("./read_BVA_plotting_tools.jl")

################################################
################################################
##           START OF MAIN PROGRAM
################################################
################################################

# Widen screen for better viewing
display("text/html", "<style>.container { width:100% !important; }</style>")

# Select a HVA or BVA file
##infil = pick_file("C:\\QGHL\\Wave_data\\Bris\\BVA\\", filterlist="HVA,BVA;hva,bva");
##infil = pick_file("C:\\QGHL\\Wave_data\\Brisbane_offshore\\", filterlist="HVA,BVA;hva,bva");
#infil = pick_file("C:\\");
infil = pick_file("G:\\Wave_data\\Card Data\\", filterlist="HVA,BVA;hva,bva");
##infil = "C:\\QGHL\\Wave_data\\Bris\\Mk4\\2019\\11\\brisbane_4183}2019-11-01T00h00Z.hva"
println("Selected ",infil)
flush(stdout)

if uppercase(split(infil, ".")[end]) == "HVA"
    
#    rename!(df,[:Sequence,:Data, :Packet]);
    
    # read data from file to df
    df = DataFrame(CSV.File(infil, header=0))

    # add column names to df as shown in DWTP Section 2.1 pp 18-19 - Status1; Data; Status2; Packet
    transform!(df, :Column2 => ByRow(x -> x[1]) => :Status1)
    transform!(df, :Column2 => ByRow(x -> x[2:end]) => :Data)
    transform!(df, :Column3 => ByRow(x -> x[1]) => :Status2)
    transform!(df, :Column3 => ByRow(x -> x[2:end]) => :Packet)

    rename!(df,:Column1 => :Sequence)
    DataFrames.select!(df, Not([:Column2, :Column3]))
            
    global df1 = deepcopy(df)
    df = handle_gaps(df)    # <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

    # identify rows where channel not received OK ( see DWTP 2.1 pp. 18-19)
    bad = findall(df[!,:Status1] .== '!')
    
    if !isempty(bad)
        
        println(length(bad)," unrecoverable records found!")
        
    else
        
        println("No unrecoverable records found!")
        
    end 

###    delete!(df, bad) # drop rows where channel is damaged beyond repair
    suspect = findall(df[!,:Status1] .== '=') # flag rows where channel is damaged but has been repaired successfully
    
    if !isempty(suspect)
        
        println(length(suspect)," damaged records found that have been repaired successfully!")
        
    else
        
        println("All records received OK!")
        
    end 

    # create a Packet vector of hex values
    packet = []
    
    for i in 1:length(df.Packet)
        
        push!(packet, string2hex(SubString(df.Packet[i],1,2)))
        push!(packet, string2hex(SubString(df.Packet[i],3,4)))
        push!(packet, string2hex(SubString(df.Packet[i],5,6)))
        
    end  
    
    # create a Data vector of hex values
    Data = lowercase.(df.Data)
    
elseif uppercase(split(infil, ".")[end]) == "BVA"
    
    #Change the type-interpretation of the binary file data to unsigned integer
    println("Reading BINARY data from ",infil)
    data = reinterpret(UInt8, read(infil))

    # turn the data vector into a matrix of 12 values matching hexadecimal bytes - see DWTP 2.1 p.18
    cols = 12
    rows = Int(length(data) / cols)
    mat = reshape(view(data, :), cols, :)

    # Interleave last 4 matrix columns to form packet vector
    ## based on mschauer @ https://discourse.julialang.org/t/combining-two-arrays-with-alternating-elements/15498/2
    packet = collect(Iterators.flatten(zip(mat[10:12,:])))
        
    ## get data for the Heave, North, and West displacements
    # Convert binary data to hexidecimal vectors
    println("Building displacements vectors - this takes a while!")
    flush(stdout)
    Data = [join([string(data[i], base = 16, pad = 2) for i = j+1 : j+12])[1:18] for j in 0:12:length(data)-12]
        
else
        
    println("Not able to read this file type at present")
    exit()
        
end
          
f23_df = DataFrame(Date = [], Segments = [], Match_vector = [], Sample_number = [])

f29_df = DataFrame(Date = [], Coverage = [], Nw = [], Epsilon = [], Hmax = [], THmax = [], H10 = [], 
    TH10 = [], H3 = [], TH3 = [], Havg = [], Tavg = [])
    
for i in 0:22
        
    col_name = "Hq$i"
    f29_df[!,col_name] = []
        
end

fc1_df = DataFrame(Date = [], Firmware = [], Hatch_uid = [], Hull_uid = [], Uptime = [], 
    Battery_energy = [], Boostcaps_energy = [],
    Hatch_temp = [], Battery_voltage = [], Batteries_per_section = [], Battery_section_number = [], 
    Initial_battery_energy = [], Ov = [], Cv = [], Ox = [], Oy = [], Cx = [], Cy = [], μ₀ = [], 
    σ₀ = [], μᵢ = [], σᵢ = [], μₕ = [], σh = [], Cpitch = [], Croll = [], Tensor = [])
    
# find all occurrences of 0x7e in packet vector
aa = findall(x->x.==0x7e, vec(packet))

# determine number of records
max_val = length(aa) - 1

for i in 1 : max_val

    # determine packet length
    first = aa[i] + 1
    last = aa[i+1]
            
    list1 = []; list2 = []
    
    if (last-first > 1)
            
        global decoded = []
        decoded = packet[first:last-1]
        decode_length = length(decoded)
                
        bb = findall(x->x.==0x7d, vec(decoded))
            
        if bb != []

            # do an xor of elements with 0x7d
            for ii in bb
                    
                if ii < (last-first)
                        
                    decoded[ii+1] = decoded[ii+1] ⊻ 0x20 # set the xor value as 0x20 vide 2.1.2 p.20
                        
                end
                    
            end

            # remove the 0x7d
            deleteat!(decoded::Vector, bb)
            decode_length = length(decoded)

        end

        if decoded[2] == 0x23
                  
            if decode_length != 22
                    
                println("Alert 0xF23 message length is ", decode_length, " but should be 22")
                    
            else

                timestamp, segments_used, match_vector, sample_number = process_f23(decoded)
                push!(f23_df, [timestamp, segments_used, match_vector, sample_number])

            end
                
        end
            
        if decoded[2] == 0x29

            if decode_length != 59
                    
                println("Alert 0xF29 message length is ", decode_length, " but should be 59")
                    
            else                 
                    
                hq = []
                timestamp, coverage, nw, epsilon, hmax, thmax, h10, th10, h3, th3, havg, tavg, hq = process_f29(decoded,hq)         
                list_1 = [timestamp, coverage, nw, epsilon, hmax, thmax, h10, th10, h3, th3, havg, tavg]
                push!(f29_df, [list_1; hq])
                    
            end
        end
            
        if decoded[2] == 0xc1

            if decode_length != 67
                    
                println("Alert 0xFC1 message length is ", decode_length, " but should be 67")
                    
            else                 
            
                timestamp, firmware, hatch_uid, hull_uid, uptime, battery_energy, boostcaps_energy, hatch_temp, battery_voltage, batteries_per_section,
                    battery_section_number, initial_battery_energy, ov, cv, ox, oy, cx, cy, μ₀, σ₀, μᵢ, σᵢ, μₕ, σh, cpitch, croll, tensor = process_fc1(decoded)   

                push!(fc1_df, [timestamp, firmware, hatch_uid, hull_uid, uptime, battery_energy, boostcaps_energy, hatch_temp, battery_voltage, batteries_per_section,
                    battery_section_number, initial_battery_energy, ov, cv, ox, oy, cx, cy, μ₀, σ₀, μᵢ, σᵢ, μₕ, σh, cpitch, croll, tensor])
                    
            end
                
        end 
            
    end
        
end        
        
f23_df = unique(f23_df)
fc1_df = unique(fc1_df)

println("All file data read!")
println("Preparing to plot data")
flush(stdout)
    
# remove those vectors from F23 df that are not located in the Data vector df
f23_df[!,"Data_vector"] = [findfirst(x->x==i, Data) for i in f23_df.Match_vector]

## Plot 30-minute records
# create a vector of dates from the F23 df
vector = Dates.format.(f23_df.Date, "yyyy-mm-dd HH:MM:SS")

# Do time-series plot of available data
plot_f29(f29_df)

println("Select date from menu for more plots")

################################################
################################################
##           END OF MAIN PROGRAM
################################################
################################################

## Filter long-period displacements in Heave, North, and West

In [ ]:
# refer to https://github.com/lnacquaroli/SavitzkyGolay.jl
using SavitzkyGolay

function get_width(wse)
################################
# determine filter's window size based on wave height       
    if maximum(wse) < 2
        
        return(21)
        
    elseif maximum(wse) < 3
        
        return(25)
        
    elseif maximum(wse) < 4.5
        
        return(29)
        
    elseif maximum(wse) < 6
        
        return(33)
        
    else
        
        return(35)
        
    end
    
    end    # get_width()
    

function do_sg(wse, plane)
################################
# apply a Savitsky Golan filter to data to remove large displacements
    
    finished = false; no_filter = true
    result = wse
    
    # locate values outside valid range and set to NaN - refer DWTP p.19 Section 2.1.1
    result[abs.(result).>=20] .= NaN    
    
    # create a filter array of zeros
    combined_filter = zeros(length(wse))
    
    # Set length of filter (based on width of Datawell's Mk4 Double Integration and Band Pass Filter )
    filter_length = 255
   
    # loop through the record until all large displacements have been processed
    while !finished

        # determine length of the filter window and apply filter to displacement
        window_size = get_width(result)                    

        # apply a Savitzky Golay filter to each occurrence of large displacements
        sg = savitzky_golay(result, window_size, 3);
        
        # locate position of the peak of each large displacement in the record
        location = findmax(x->isnan(x) ? 0 : x,sg.y)[2]

        # use the following condition as test for large displacement
        ##############################################################################################################
        ## keep entering the "if" statement if the sg peak is greater than 3m and it is more than 5 times the mean WSE  
        ##############################################################################################################
        if (result[location] > 3) && (result[location] > 5 * mean(filter(!isnan, abs.(result))))

            # print spike details to screen
            @printf("%s spike at %s  height = %4.2fm; Mean = %4.2fm; Median =  %4.2fm\n", plane, 
                Dates.format(times[location], "dd-mm-yyyy HH:MM"), result[location], mean(filter(!isnan, 
                abs.(result))), median(filter(!isnan, abs.(result))))

            # find where peak in savitzky_golay curve
            location = findmax(x->isnan(x) ? 0 : x,sg.y)[2]

            # set lower and upper limits of savitzky_golay filter
            ## this handles situations where large displacements occur within filter_length of the start or end
            ## of the record.
            if location-filter_length >= 1

                lower = location - filter_length

            else

                lower = 1

            end

            if location+filter_length < length(result)

                upper = location + filter_length

            else

                upper = length(result)

            end

            # set savitzky_golay filter to zero outside limits
            ## this means that filter is only applied to affected data (and not elsewhere)
            sg.y[1:lower] .= 0
            sg.y[upper:end] .= 0
            result -= sg.y

            # build a combined filter from multiple events in a record
            combined_filter += sg.y
                        
        else
            
            # set flag to exit loop
            finished = true
            
        end
        
    end
    
    return(combined_filter)
                    
    end     # do_sg()                  
                        
outfil = rsplit(infil, "\\"; limit=2)[2][1:end-4] 

# Set length of filter
filter_length = 200

# Convert datetime array to string array
date_df = f23_df[2:end,:]
dates = Dates.format.(date_df.Date, "yyyy-mm-dd HH:MM:SS");

w = Toplevel("Select Date", 235, 600)
tcl("pack", "propagate", w, false)
f = Frame(w)
pack(f, expand=true, fill="both")

f1 = Frame(f)
lb = Treeview(f1, dates)
scrollbars_add(f1, lb)
pack(f1,  expand=true, fill="both")

tcl("ttk::style", "configure", "TButton", foreground="blue", font="arial 16 bold")
b = Button(f, "Ok")
pack(b)

bind(b, "command") do path
    
    date_choice = get_value(lb)
    global idx = indexin(date_choice, dates)
    idx = idx[1]
    

    global start_date, start_val, end_val = get_start_end_dates(f23_df, idx+2)
    title_string = Dates.format(start_date, "dd/mm/yyyy HH:MM")
    global heave, north, west = get_hnw(Data, start_val, end_val);
    
    # set time stamps for each record sample                         
    global times = julian2datetime.(datetime2julian(start_date) .+ collect(0:1:length(heave)-1)/2.56 ./ 86400)                            

    # filter heave, west, and west records
    global heave_filter = do_sg(heave, "Heave") 
    global north_filter = do_sg(north, "North")   
    global west_filter = do_sg(west, "West")
#==    
    if heave_filter[abs.(heave_filter)] > 20 heave = NaN
    north_filter[abs.(north_filter) .> 20] .= NaN
    west_filter[abs.(west_filter) .> 20] .= NaN
==#
    # plot results
    p1 = plot(times, heave, label="Heave", title="Heave", titlefontsize=:10, lw=0.5, lc=:grey)
    p1 = plot!(times, heave-heave_filter, label="Filtered", lc=:red)
    
    # get plotting limits, and display input file name at top left of plot
    x_lim1 = xlims(p1)[1]; y_lim1 = ylims(p1)[1]
    x_lim2 = xlims(p1)[2]; y_lim2 = ylims(p1)[2]   
    x_pos = x_lim1 + abs(x_lim2-x_lim1)*0.04; y_pos = y_lim2*1.1
    file_name_string = replace(infil, "\\" => "/")
    p1 = annotate!(x_pos, y_pos, text("  "*file_name_string, :red, :left, 8))

    x_pos = x_lim1 + abs(x_lim2-x_lim1)*0.04; y_pos = y_lim2*0.9
    p1 = annotate!(x_pos, y_pos, text("Firmwave ver. = " * fc1_df.Firmware[1], :grey, :left, 7))
    x_pos = x_lim1 + abs(x_lim2-x_lim1)*0.14
    p1 = annotate!(x_pos, y_pos, text("Hatch UID = " * string(fc1_df.Hatch_uid[1]), :grey, :left, 7))
    x_pos = x_lim1 + abs(x_lim2-x_lim1)*0.24
    p1 = annotate!(x_pos, y_pos, text("Hull UID = " * string(fc1_df.Hull_uid[1]), :grey, :left, 7))    

    p2 = plot(times, north, label="North", title="North", titlefontsize=:10, lw=0.5, lc=:grey)
    p2 = plot!(times, north-north_filter, label="Filtered", lc=:green)

    p3 = plot(times, west, label="West", title="West", titlefontsize=:10, lw=0.5, lc=:grey)
    p3 = plot!(times, west-west_filter, label="Filtered", lc=:blue)
    
    # display plots to screen
    global tm_tick = range(first(times), last(times), step=Minute(5))
    global ticks = Dates.format.(tm_tick,"dd-mm-yy HH:MM")

    plot_heave = plot(p1, p2, p3, layout=(3,1), size = (1800,800), fg_legend=:transparent, bg_legend=:transparent,
        xticks=(tm_tick,ticks), xlim=(first(times), last(times)), xminorgrid=:true, xminorticks=:5, 
        bottommargin=2*Plots.mm, legend=:topright, ylabel="", framestyle=:box)

    # save plots to .PNG file
    savefig(".\\"*rsplit(infil, "\\"; limit=3)[2] * '_' * rsplit(infil, "\\"; limit=3)[3][1:end-4]*Dates.format(start_date, "_HHMM")*".png")

    display(plot_heave)
    
    global spectra_heave = welch_pgram(map(x -> isnan(x) ? zero(x) : x, heave), 512, 256; onesided=true, 
        nfft=512, fs=2.56, window=hanning)
    global spectra_north = welch_pgram(map(x -> isnan(x) ? zero(x) : x, north), 512, 256; onesided=true, 
        nfft=512, fs=2.56, window=hanning)
    global spectra_west = welch_pgram(map(x -> isnan(x) ? zero(x) : x, west), 512, 256; onesided=true, 
        nfft=512, fs=2.56, window=hanning)

    spectra_heave_filter = welch_pgram(map(x -> isnan(x) ? zero(x) : x, heave-heave_filter), 512, 256; onesided=true, 
        nfft=512, fs=2.56, window=hanning)
    spectra_north_filter = welch_pgram(map(x -> isnan(x) ? zero(x) : x, north-north_filter), 512, 256; onesided=true, 
        nfft=512, fs=2.56, window=hanning)
    spectra_west_filter = welch_pgram(map(x -> isnan(x) ? zero(x) : x, west-west_filter), 512, 256; onesided=true, 
        nfft=512, fs=2.56, window=hanning)
    
    Tp_heave = 1/freq(spectra_heave)[findmax(power(spectra_heave))[2]]
    Tp_north = 1/freq(spectra_north)[findmax(power(spectra_north))[2]]
    Tp_west = 1/freq(spectra_west)[findmax(power(spectra_west))[2]]
        
    Tp_heave_filter = 1/freq(spectra_heave_filter)[findmax(power(spectra_heave_filter))[2]]
    Tp_north_filter = 1/freq(spectra_north_filter)[findmax(power(spectra_north_filter))[2]]
    Tp_west_filter = 1/freq(spectra_west_filter)[findmax(power(spectra_west_filter))[2]]
                            
    s1 = plot(freq(spectra_heave), power(spectra_heave), xlim=(0,0.64), ylim=(floor(minimum(power(spectra_heave))), 
        ceil(maximum(power(spectra_heave)))), title="Heave", titlefontsize=10, lw=:1, lc=:red, ls=:dash, 
        label="Tp = "*string(round.(Tp_heave; digits=2))*"s\n",fillrange=0, fillalpha=0.025, fillcolor=:red)
    s1 = plot!(freq(spectra_heave_filter), power(spectra_heave_filter), lw=:2, lc=:blue, 
        label="Tp_heave_filtered = "*string(round.(Tp_heave_filter; digits=2))*"s", fillrange=0, fillalpha=0.075, 
        fillcolor=:blue)
    s1 = vline!([freq(spectra_heave)[findmax(power(spectra_heave))[2]]], lw=1, lc=:red, ls=:dash, label="")                            
    s1 = vline!([freq(spectra_heave_filter)[findmax(power(spectra_heave_filter))[2]]], lw=1, lc=:blue, ls=:dash, 
        label="")                                                        
                            
    s2 = plot(freq(spectra_north), power(spectra_north), xlim=(0,0.64), ylim=(floor(minimum(power(spectra_north))), 
        ceil(maximum(power(spectra_north)))), title="North", titlefontsize=10, lw=:1, lc=:red, ls=:dash, 
        label="Tp = "*string(round.(Tp_north; digits=2))*"s\n",fillrange=0, fillalpha=0.025, fillcolor=:red)
    s2 = plot!(freq(spectra_north_filter), power(spectra_north_filter), lw=:2, lc=:blue, 
        label="Tp_north_filtered = "*string(round.(Tp_north_filter; digits=2))*"s", fillrange=0, fillalpha=0.075, 
        fillcolor=:blue)
    s2 = vline!([freq(spectra_north)[findmax(power(spectra_north))[2]]], lw=1, lc=:red, ls=:dash, label="")                            
    s2 = vline!([freq(spectra_north_filter)[findmax(power(spectra_north_filter))[2]]], lw=1, lc=:blue, ls=:dash, 
        label="")     
                            
    s3 = plot(freq(spectra_west), power(spectra_west), xlim=(0,0.64), ylim=(floor(minimum(power(spectra_west))), 
        ceil(maximum(power(spectra_west)))), title="West", titlefontsize=10, lw=:1, lc=:red, ls=:dash, 
        label="Tp = "*string(round.(Tp_west; digits=2))*"s\n",fillrange=0, fillalpha=0.025, fillcolor=:red)
    s3 = plot!(freq(spectra_west_filter), power(spectra_west_filter), lw=:2, lc=:blue, 
        label="Tp_west_filtered = "*string(round.(Tp_west_filter; digits=2))*"s", fillrange=0, fillalpha=0.075, 
        fillcolor=:blue)
    s3 = vline!([freq(spectra_west)[findmax(power(spectra_west))[2]]], lw=1, lc=:red, ls=:dash, label="")                            
    s3 = vline!([freq(spectra_west_filter)[findmax(power(spectra_west_filter))[2]]], lw=1, lc=:blue, ls=:dash, label="")                              
                            
    plot_spectra = plot(s1, s2, s3, layout=(1,3), size = (1800,600), fg_legend=:transparent, bg_legend=:transparent,
        legend=:topright, bottommargin=10Plots.mm, leftmargin=0Plots.mm, ylabel="", framestyle = :box)                            
      
    display(plot_spectra)

    d2_1 = plot(-west,north, zcolor=heave, m=(1, 3, :RdYlGn_11, Plots.stroke(0)), leg=false, cbar=false, c="lightgrey", 
        colorbar_title="Heave displacement (m)", w=0.5, xlabel="East-West displacement (m)", 
        ylabel="North-South displacement (m)", title = "2d-plot before filter", aspect_ratio=:equal, 
        xlim=(minimum(-west)*1.1, maximum(-west)*1.1), ylim=(minimum(north)*1.1,maximum(north)*1.1))

    d2_2 = plot(-(west-west_filter),north-north_filter, zcolor=heave, m=(1, 3, :RdYlGn_11, Plots.stroke(0)), 
        leg=false, cbar=false, c="lightgrey", colorbar_title="Heave displacement (m)", w=0.5, 
        xlabel="East-West displacement (m)", ylabel="North-South displacement (m)", title = "2d-plot after filter", 
        aspect_ratio=:equal, xlim=(minimum(-(west-west_filter))*1.1, maximum(-(west-west_filter))*1.1), 
        ylim=(minimum(north-north_filter)*1.1,maximum(north-north_filter)*1.1))                                

    plot_2d = Plots.plot(d2_1, d2_2, layout = (1, 2), size = (1300, 600), leftmargin=15Plots.mm, rightmargin=1Plots.mm,
        topmargin=1Plots.mm, bottommargin=15Plots.mm, titlefontsize=:10, grid=true, gridlinewidth=0.5, gridstyle=:dot, 
        gridalpha=1, framestyle = :box) 

    # display 2d plots to screen
    display(plot_2d)

    # create 3d plots of heave, north, and west
    d3_1 = plot(-west, north, heave, zcolor=heave, m=(1, 3, :RdYlGn_11, Plots.stroke(0)), leg=false, lc=:grey, lw=:0.5,
        aspect_ratio=:equal, xlabel="East-West", title = "3d-plot before filter", ylabel="North-South", zlabel="Heave",
        framestyle = :box)

    d3_2 = plot(-(west-west_filter), north-north_filter, heave-heave_filter, zcolor=heave-heave_filter, 
        m=(1, 3, :RdYlGn_11, Plots.stroke(0)), leg=false, lc=:grey, lw=:0.5, aspect_ratio=:equal,
        title = "3d-plot after filter", xlabel="East-West", ylabel="North-South", zlabel="Heave", framestyle = :box)
                            
    plot_3d = Plots.plot(d3_1, d3_2, layout = (1, 2), size = (1300, 600), titlefontsize=:10);         

    # display 3d plots to screen
    display(plot_3d)                            
    
end

In [ ]:
function calc_spectrogram(wse)
# calculate spectrogram
    spec = DSP.Periodograms.spectrogram(wse, 128, 120; fs=2.56,window=hanning)
end

spec_h = calc_spectrogram(heave)
spec_hf = calc_spectrogram(heave.-heave_filter)
spec_n = calc_spectrogram(north)
spec_nf = calc_spectrogram(north.-north_filter)
spec_w = calc_spectrogram(west)
spec_wf = calc_spectrogram(west.-west_filter)

# display plots to screen
tm_tick = range(start_date, start_date + Minute(30), step=Minute(5))
ticks = Dates.format.(tm_tick,"HH:MM:SS")

# set limits of y-range in plots
ylim = (0,0.3)

# get spectrogram time values
times = julian2datetime.(datetime2julian(start_date) .+ spec_h.time ./ 86400)

sp_h = plot(times, spec_h.freq, DSP.Periodograms.power(spec_h), lw=0.5, xlabel="", xlim=(start_date, start_date + Minute(30)), xticks=(tm_tick,ticks), xtickfontsize=7, xminorgrid=:true, xminorticks=:5,
    ylabel="Frequency (Hz)", ylim=ylim, ytickfontsize=8, 
    title="Heave " * Dates.format(start_date, "dd/mm/yyyy HH:MM"), framestyle = :box,
    leftmargin = 15Plots.mm, bottommargin = 15Plots.mm, grid=true, size=(1800, 600), colorbar=false, gridlinewidth=0.5, gridstyle=:dot, gridalpha=1)

# get spectrogram time values
times = julian2datetime.(datetime2julian(start_date) .+ spec_hf.time ./ 86400)

sp_hf = plot(times, spec_hf.freq, DSP.Periodograms.power(spec_hf), lw=0.5, xlabel="", xlim=(start_date, start_date + Minute(30)), xticks=(tm_tick,ticks), xtickfontsize=7, xminorgrid=:true, xminorticks=:5,
    ylabel="Frequency (Hz)", ylim=ylim, ytickfontsize=8, 
    title="Heave filtered " * Dates.format(start_date, "dd/mm/yyyy HH:MM"), framestyle = :box,
    leftmargin = 15Plots.mm, bottommargin = 15Plots.mm, grid=true, size=(1800, 600), colorbar=false, gridlinewidth=0.5, gridstyle=:dot, gridalpha=1)

sp_n = plot(times, spec_n.freq, DSP.Periodograms.power(spec_n), lw=0.5, xlabel="", xlim=(start_date, start_date + Minute(30)), xticks=(tm_tick,ticks), xtickfontsize=7, xminorgrid=:true, xminorticks=:5,
    ylabel="Frequency (Hz)", ylim=ylim, ytickfontsize=8, 
    title="North " * Dates.format(start_date, "dd/mm/yyyy HH:MM"), framestyle = :box,
    leftmargin = 15Plots.mm, bottommargin = 15Plots.mm, grid=true, size=(1800, 600), colorbar=false, gridlinewidth=0.5, gridstyle=:dot, gridalpha=1)

# get spectrogram time values
times = julian2datetime.(datetime2julian(start_date) .+ spec_nf.time ./ 86400)

sp_nf = plot(times, spec_nf.freq, DSP.Periodograms.power(spec_nf), lw=0.5, xlabel="", xlim=(start_date, start_date + Minute(30)), xticks=(tm_tick,ticks), xtickfontsize=7, xminorgrid=:true, xminorticks=:5,
    ylabel="Frequency (Hz)", ylim=ylim, ytickfontsize=8, 
    title="North filtered " * Dates.format(start_date, "dd/mm/yyyy HH:MM"), framestyle = :box,
    leftmargin = 15Plots.mm, bottommargin = 15Plots.mm, grid=true, size=(1800, 600), colorbar=false, gridlinewidth=0.5, gridstyle=:dot, gridalpha=1)

sp_w = plot(times, spec_w.freq, DSP.Periodograms.power(spec_w), lw=0.5, xlabel="", xlim=(start_date, start_date + Minute(30)), xticks=(tm_tick,ticks), xtickfontsize=7, xminorgrid=:true, xminorticks=:5,
    ylabel="Frequency (Hz)", ylim=ylim, ytickfontsize=8, 
    title="West " * Dates.format(start_date, "dd/mm/yyyy HH:MM"), framestyle = :box,
    leftmargin = 15Plots.mm, bottommargin = 15Plots.mm, grid=true, size=(1800, 600), colorbar=false, gridlinewidth=0.5, gridstyle=:dot, gridalpha=1)

# get spectrogram time values
times = julian2datetime.(datetime2julian(start_date) .+ spec_wf.time ./ 86400)

sp_wf = plot(times, spec_wf.freq, DSP.Periodograms.power(spec_wf), lw=0.5, xlabel="", xlim=(start_date, start_date + Minute(30)), xticks=(tm_tick,ticks), xtickfontsize=7, xminorgrid=:true, xminorticks=:5,
    ylabel="Frequency (Hz)", ylim=ylim, ytickfontsize=8, 
    title="West filtered " * Dates.format(start_date, "dd/mm/yyyy HH:MM"), framestyle = :box,
    leftmargin = 15Plots.mm, bottommargin = 15Plots.mm, grid=true, size=(1800, 600), colorbar=false, gridlinewidth=0.5, gridstyle=:dot, gridalpha=1)


sp_plot = plot(sp_h, sp_hf, sp_n, sp_nf, sp_w, sp_wf, layout=(3,2), size=(1800,900), leftmargin=10Plots.mm, bottommargin=5Plots.mm, ylabelfontsize=:8, titlefontsize=:10)

display(sp_plot)

In [ ]:
p1 = plot(north[1:100], label="", marker=:circle, size=(1800,600))
p1 = plot!(north_filter[1:100], label="", marker=:cross)

for i in 1:2:100
    p1 = vline!([i], lc=:grey, ls=:dash, alpha=:0.25, label="")
end

display(p1)

In [ ]:
plot(north_filter, size=(1800,600), label="Filter")
plot!(north, label="Heave")
plot!(north .- north_filter, label="Result")

In [ ]:
plot(north[1:200])
if abs.(north[1:200] .- north_filter[1:200]) .> 2 
    north_filter[1:200] .= NaN 
end
plot!(north_filter[1:200])    

## Working copy of correcting 30-minute record using SavitzkyGolay

In [ ]:
# refer to https://github.com/lnacquaroli/SavitzkyGolay.jl
using SavitzkyGolay

##gr() 


function get_width(displacement)
################################
        
    if maximum(displacement) < 2
        
        return(21)
        
    elseif maximum(displacement) < 3
        
        return(25)
        
    elseif maximum(displacement) < 4.5
        
        return(29)
        
    elseif maximum(displacement) < 6
        
        return(33)
        
    else
        
        return(35)
        
    end
    
    end    # get_width()
    
# Set length of filter
filter_length = 200

# Convert datetime array to string array
date_df = f23_df[2:end,:]
dates = Dates.format.(date_df.Date, "yyyy-mm-dd HH:MM:SS");

w = Toplevel("Select Date", 235, 600)
tcl("pack", "propagate", w, false)
f = Frame(w)
pack(f, expand=true, fill="both")

f1 = Frame(f)
lb = Treeview(f1, dates)
scrollbars_add(f1, lb)
pack(f1,  expand=true, fill="both")

tcl("ttk::style", "configure", "TButton", foreground="blue", font="arial 16 bold")
b = Button(f, "Ok")
pack(b)

bind(b, "command") do path
    
    date_choice = get_value(lb)
    global idx = indexin(date_choice, dates)
    idx = idx[1]
    
    start_date, start_val, end_val = get_start_end_dates(f23_df, idx+2)
    heave, north, west = get_hnw(Data, start_val, end_val);

    # time stamp each WSE
    points = collect(0:1:length(heave)-1) / 2.56
    times = []

    for i in 1 : length(points)
        
        push!(times, unix2datetime(datetime2unix(start_date) + points[i]))
            
    end

    # create plots of heave, north, and west
    title_string = Dates.format(start_date, "dd/mm/yyyy HH:MM")

    global displacement = heave
    
    # time stamp each WSE
    points = collect(0:1:length(displacement)-1)/2.56
    times = []

    for i in 1:length(points)
        push!(times,unix2datetime(datetime2unix(start_date) + points[i]))
    end

    finished = false; no_filter = true
    result = displacement
##    result[abs.(result).>=20] .= NaN        
    combined_filter = zeros(4608)
    outfil = rsplit(infil, "\\"; limit=2)[2][1:end-4]        

    while !finished

        # determine length of the filter window
        if maximum(result) < 2
            window_size = 21
        elseif maximum(result) < 3
            window_size = 25
        elseif maximum(result) < 4
            window_size = 29
        elseif maximum(result) < 6
            window_size = 33
        else
            window_size = 41
        end

        # determine length of the filter window and apply filter to displacement
        window_size = get_width(result)            
        global sg = savitzky_golay(result, window_size, 3);
            
#vvvvvvv lines below are work in progress vvvvvvv
        location = findmax(sg.y)[2]
##        println("Spike height = ",result[location]," Mean = ",mean(abs.(result))," Median = ",median(abs.(result)))
        
#^^^^^^^ lines above are work in progress #^^^^^^^
        # calculate spectrum of filter and locate peak frequency
        spectra_sg = welch_pgram(sg.y, 512, 256; onesided=true, nfft=512, fs=2.56, window=hanning)
        fp_sg = freq(spectra_sg)[findmax(power(spectra_sg))[2]]
        Tp1 = 1/fp_sg

        # loop while Tp > 20
##        if Tp1 > 20
        if result[location] > 5 * mean(abs.(result))
            
            no_filter = false

            # find where peak in savitzky_golay curve
            location = findmax(sg.y)[2]

            # set lower and upper limits of savitzky_golay filter
            if location-filter_length > 1

                lower = location - filter_length

            else

                lower = 1

            end

            if location+255 < length(result)

                upper = location + filter_length

            else

                upper = length(result)

            end

            # set savitzky_golay filter to zero outside limits
            sg.y[1:lower] .= 0
            sg.y[upper:end] .= 0
            result -= sg.y

            # keep record of combined filter
            combined_filter += sg.y

            # display plots to screen
            global tm_tick = range(first(times), last(times), step=Minute(5))
            global ticks = Dates.format.(tm_tick,"HH:MM")

            xpos = times[1]
            ypos = maximum(displacement)

            global pp1 = plot(times, displacement, label="Uncorrected WSE", lc=:yellow,
                title=outfil*" - "*title_string, titlefontsize=12)
            pp1 = plot!(times, combined_filter, label="Filter", lc=:red, ls=:dot)
            pp1 = vline!([times[location]], lw=1, lc=:red, ls=:dash, label="")
            pp1 = annotate!(xpos, ypos, text("  "*replace(infil, "\\" => "/"), :red, :left, 8))

            # get plotting limits
            x_lim1 = xlims(pp1)[1]; y_lim1 = ylims(pp1)[1]
            x_lim2 = xlims(pp1)[2]; y_lim2 = ylims(pp1)[2]

            global x_pos = x_lim1 + abs(x_lim2-x_lim1)*0.02
            pp1 = annotate!(x_pos, y_lim2*1.025, text("Firmwave ver. = " * fc1_df.Firmware[1], :grey, :left, 7))
            x_pos = x_lim1 + abs(x_lim2-x_lim1)*0.13
            pp1 = annotate!(x_pos, y_lim2*1.025, text("Hatch UID = " * string(fc1_df.Hatch_uid[1]), :grey, :left, 7))
            x_pos = x_lim1 + abs(x_lim2-x_lim1)*0.26
            pp1 = annotate!(x_pos, y_lim2*1.025, text("Hull UID = " * string(fc1_df.Hull_uid[1]), :grey, :left, 7))
            pp1 = plot!(times, result, lw=1, lc=:blue, label="Corrected WSE")     

        else

            finished = true
            
            if no_filter
                
                # display plots to screen
                global tm_tick = range(first(times), last(times), step=Minute(5))
                global ticks = Dates.format.(tm_tick,"HH:MM")

                xpos = times[1]
                ypos = maximum(displacement)
                
                pp1 = plot(times, displacement, label="Uncorrected WSE", lc=:blue, title=outfil*" - "*title_string, 
                    titlefontsize=12)
                
                 # get plotting limits
                x_lim1 = xlims(pp1)[1]; y_lim1 = ylims(pp1)[1]
                x_lim2 = xlims(pp1)[2]; y_lim2 = ylims(pp1)[2]

                global x_pos = x_lim1 + abs(x_lim2-x_lim1)*0.02
                pp1 = annotate!(x_pos, y_lim2*1.025, text("Firmwave ver. = " * fc1_df.Firmware[1], :grey, :left, 7))
                x_pos = x_lim1 + abs(x_lim2-x_lim1)*0.13
                pp1 = annotate!(x_pos, y_lim2*1.025, text("Hatch UID = " * string(fc1_df.Hatch_uid[1]), :grey, :left, 7))
                x_pos = x_lim1 + abs(x_lim2-x_lim1)*0.26
                pp1 = annotate!(x_pos, y_lim2*1.025, text("Hull UID = " * string(fc1_df.Hull_uid[1]), :grey, :left, 7))
                pp1 = plot!(times, result, lw=1, lc=:blue, label="Corrected WSE") 
                
                plot_p1 = plot(pp1, layout=(1,1), size = (1800,500), xlim=(first(times),last(times)), 
                    xticks=(tm_tick,ticks), fg_legend=:transparent, legend=:bottomleft, 
                    bottommargin=10Plots.mm, leftmargin=0Plots.mm, ylabel="", framestyle = :box)

            else

                plot_p1 = plot(pp1, layout=(1,1), size = (1800,500), xlim=(first(times),last(times)),
                    xticks=(tm_tick,ticks), fg_legend=:transparent, legend=:bottomleft, 
                    bottommargin = 10Plots.mm, leftmargin = 0Plots.mm, ylabel="", framestyle = :box)
                
            end

            display(plot_p1)

            spectra1w = welch_pgram(displacement, 512, 256; onesided=true, nfft=512, fs=2.56, window=hanning)
            spectra1_sg = welch_pgram(sg.y, 512, 256; onesided=true, nfft=512, fs=2.56, window=hanning)
            spectra3w = welch_pgram(result, 512, 256; onesided=true, nfft=512, fs=2.56, window=hanning)

            Tp1 = 1/freq(spectra1w)[findmax(power(spectra1w))[2]]
            Tp3 = 1/freq(spectra3w)[findmax(power(spectra3w))[2]]    

            #s1 = plot(freq(spectra1), power(spectra1), label="", framestyle = :box)
            s1 = plot(freq(spectra1w), power(spectra1w), xlim=(0,0.64), ylim=(floor(minimum(power(spectra1w))),
                ceil(maximum(power(spectra1w)))), title=outfil*" - "*title_string*" Uncorrected", titlefontsize=10,
                lw=:2, lc=:red, label="Uncorrected spectra\n",fillrange=0, fillalpha=0.075, fillcolor=:red, 
                framestyle = :box)
            s1 = vline!([freq(spectra1w)[findmax(power(spectra1w))[2]]], lw=1, lc=:grey, ls=:dash,
                label="Tp = "*string(round.(Tp1; digits=2))*"s")

            s3 = plot(freq(spectra1w), power(spectra1w), xlim=(0,0.64), ylim=(floor(minimum(power(spectra1w))),
                ceil(maximum(power(spectra1w)))), title=outfil*" - "*title_string*" Corrected", titlefontsize=10,        
                lw=:1, lc=:red, ls=:dash, label="",fillrange=0, fillalpha=0.025, fillcolor=:red)
            s3 = plot!(freq(spectra3w), power(spectra3w), lw=:2, lc=:blue, label="Corrected spectra\n",
                fillrange=0, fillalpha=0.075, fillcolor=:blue)
            s3 = vline!([freq(spectra3w)[findmax(power(spectra3w))[2]]], lw=1, lc=:grey, ls=:dash, 
                label="Tp = "*string(round.(Tp3; digits=2))*"s")

            plot_s1 = plot(s1, s3, layout=(1,2), size = (1800,600), fg_legend=:transparent, bg_legend=:transparent,
                legend=:topright, bottommargin=10Plots.mm, leftmargin=0Plots.mm, ylabel="", framestyle = :box)

            # save a copy of plot as a .PNG file
    ##        savefig(".\\"*outfil*"_filtered_Spectra"*Dates.format(start_date, "_HHMM")*".png")

            display(plot_s1)

        end

    end
    
end

In [ ]:
found_list = idx + 2

start_date = f23_df[found_list[1],:].Date - Minute(30) # <------- NOTE subtracted 30min from start_date to match Waves4 results
segments = f23_df[found_list[1],:].Segments
#   match_vector = f23_df[found_list[1],:].Match_vector
sample_nos = f23_df[found_list[1],:].Sample_number
data_vector = f23_df[found_list[1],:].Data_vector
start_val = data_vector - Int(sample_nos/2) + 1
end_val = data_vector

In [ ]:
north[abs.(north).> 10] .= 0

plot(north, size=(1800,600))
plot!(north1)

sg = savitzky_golay(north1, 31, 3);

In [ ]:
plot(north, size=(1800,600))
#plot!(north1)

#plot!(north1-sg.y)

## Plot selected 30-minute record

In [ ]:
function plot_hnw(f23_df,fc1_df,Data,idx)
######################################## 

    function spike_value(wse)
    #####################################    
        median_value = median(wse)
        std_value = std(wse)
        
        return(median_value + 3*std_value)
        
        end    # spike_value()


    println("Preparing to plot heave, north, and west time series")
    # Extract parameters from F23 df
    start_date, start_val, end_val = get_start_end_dates(f23_df,idx)

    # get WSEs for desired 30-minute record
    heave, north, west = get_hnw(Data,start_val,end_val)

    spike = spike_value(heave)
    heave_spikes = findall(i->(i>=spike), abs.(heave));

    spike = spike_value(north)
    north_spikes = findall(i->(i>=spike), abs.(north));

    spike = spike_value(west)
    west_spikes = findall(i->(i>=spike), abs.(west));

    # time stamp each WSE
    points = collect(0:1:length(heave)-1)/2.56
    times = []

    for i in 1:length(points)
        push!(times,unix2datetime(datetime2unix(start_date) + points[i]))
    end

    # create plots of heave, north, and west
    title_string = Dates.format(start_date, "dd/mm/yyyy HH:MM")
    p1_hnw = scatter(times[heave_spikes], heave[heave_spikes], label="", markershape=:circle, ms=4, mc=:white, ma=1, msc=:red, msa=0.25, msw=0.5)
    p1_hnw = plot!(times,heave, label="", c="#4a536b", lw=0.5, title=title_string, titlefontsize=12) ##last(split(infil,"\\")))

    # get plotting limits
    x_lim1 = xlims(p1_hnw)[1]; y_lim1 = ylims(p1_hnw)[1]
    x_lim2 = xlims(p1_hnw)[2]; y_lim2 = ylims(p1_hnw)[2]

    x_pos = x_lim1 + abs(x_lim2-x_lim1)*0.02
    p1_hnw = annotate!(x_pos, y_lim2*1.1, text("Firmwave ver. = " * fc1_df.Firmware[1], :grey, :left, 7))
    x_pos = x_lim1 + abs(x_lim2-x_lim1)*0.13
    p1_hnw = annotate!(x_pos, y_lim2*1.1, text("Hatch UID = " * string(fc1_df.Hatch_uid[1]), :grey, :left, 7))
    x_pos = x_lim1 + abs(x_lim2-x_lim1)*0.26
    p1_hnw = annotate!(x_pos, y_lim2*1.1, text("Hull UID = " * string(fc1_df.Hull_uid[1]), :grey, :left, 7))

    p2_hnw = scatter(times[north_spikes], north[north_spikes], label="", markershape=:circle, ms=4, mc=:white, ma=1, msc=:red, msa=0.25, msw=0.5)
    p2_hnw = plot!(times,north, label="", c="#aed6dc", lw=0.5)
    p3_hnw = scatter(times[west_spikes], west[west_spikes], label="", markershape=:circle, ms=4, mc=:white, ma=1, msc=:red, msa=0.25, msw=0.5)
    p3_hnw = plot!(times,west, label="", c="#ff9a8d", lw=0.5)

    hline!(p1_hnw, [0], lw=1, label="")
    hline!(p2_hnw, [0], lw=1, label="")
    hline!(p3_hnw, [0], lw=1, label="")

    # get plotting limits
    x_lim1 = xlims(p1_hnw)[1]; y_lim1 = ylims(p1_hnw)[1]
    x_lim2 = xlims(p1_hnw)[2]; y_lim2 = ylims(p1_hnw)[2]

    # display plots to screen
    plot_wse = Plots.plot(p1_hnw, p2_hnw, p3_hnw, layout = (3, 1), size = (1400, 900),
        xlim=(first(times),last(times)),  xticks = first(times):Minute(5):last(times),xtickfontsize=7,ytickfontsize=8,
        framestyle = :box,fg_legend=:transparent, legend=:bottomleft,
        margin = 1Plots.mm, grid=true, gridlinewidth=0.5, gridstyle=:dot, gridalpha=1)            

    display(plot_wse)

    # create a plot file to be saved as a .PNG
##    plt_file = first(infil, length(infil)-4)*"_plot_hnw_"*Dates.format(start_date, "yyyy_mm_dd_HHMM")*".png"

    # Save plot to file
##    savefig(plt_file)
##    println("Plot file saved as ",plt_file)
       
end    # plot_hnw()


# Convert datetime array to string array
date_df = f23_df[2:end,:]
dates = Dates.format.(date_df.Date, "yyyy-mm-dd HH:MM:SS");

w = Toplevel("Select Date", 235, 600)
tcl("pack", "propagate", w, false)
f = Frame(w)
pack(f, expand=true, fill="both")

f1 = Frame(f)
lb = Treeview(f1, dates)
scrollbars_add(f1, lb)
pack(f1,  expand=true, fill="both")

tcl("ttk::style", "configure", "TButton", foreground="blue", font="arial 16 bold")
b = Button(f, "Ok")
pack(b)

bind(b, "command") do path
    
    date_choice = get_value(lb)
    global idx = indexin(date_choice,dates)
    idx = idx[1]
    println(idx,' ',date_choice,' ',date_df.Date[idx])
    plot_hnw(date_df, fc1_df[2:end,:], Data, idx+1)

end

## Sleeping code to read 30-minute record and correct using SavitzkyGolay 

## How to create a 30-minute array of times incrementing by 2.56 Hertz (Mk4 sample frequency)!

In [ ]:
@time begin
    
    times = julian2datetime.(datetime2julian(start_date) .+ collect(0:1:length(heave)-1)/2.56 ./ 86400)
    
end

In [ ]:
collect(0:1:length(heave)-1)/2.56

In [ ]:
@time begin

    # time stamp each WSE
    points = collect(0:1:length(heave)-1)/2.56
    times = []

    for i in 1:length(points)
        push!(times,unix2datetime(datetime2unix(start_date) + points[i]))
    end

end

In [ ]:
filter_length = 512

aaa = north[1:4000]

window_size = get_width(aaa)

sg = savitzky_golay(aaa, window_size, 3);

# calculate spectrum of filter and locate peak frequency
spectra_sg = welch_pgram(sg.y, 512, 256; onesided=true, nfft=512, fs=2.56, window=hanning)
fp_sg = freq(spectra_sg)[findmax(power(spectra_sg))[2]]
Tp1 = 1/fp_sg

location = findmax(sg.y)[2]
println(location)

# set lower and upper limits of savitzky_golay filter
if location-filter_length > 1

    lower = location - filter_length

else

    lower = 1

end

if location+255 < length(aaa)

    upper = location + filter_length

else

    upper = length(aaa)

end

# set savitzky_golay filter to zero outside limits
sg.y[1:lower] .= 0
sg.y[upper:end] .= 0;


In [ ]:
plot(north, size=(1800,600))
plot!(aaa-sg.y)

#bbb = aaa - sg.y

In [ ]:
# 
west[west .== -2048]

In [ ]:
findfirst(contains("f80"),Data)

In [ ]:
Data[findfirst(contains("f80"),Data)]

In [ ]:
string(-2048; base=16)

In [ ]:
"f800" |> hex2bytes

In [ ]:
north[north .== missing] .= north[north .< -20.011] .== missing

In [ ]:
west[findall(abs.(west) .> 10)]

In [ ]:
plot(west)

In [ ]:
collect(Iterators.flatten(zip(SubString.(Data, start_val, end_val),SubString.(Data, start_val+9, end_val+9))));

In [ ]:
collect(Iterators.flatten(zip(SubString.(Data, start_val, end_val),SubString.(Data, start_val+9, end_val+9))))

In [ ]:
zip(SubString.(Data, start_val, end_val),SubString.(Data, start_val+9, end_val+9))

In [ ]:
SubString.(Data, start_val, end_val-1)

In [ ]:
Data[start_val:end_val,:]

In [ ]:
maximum(get_hnw(Data,start_val,end_val)[3])

In [ ]:
maximum(0.457-sinh((3893-2048)/457))

In [ ]:
data = reinterpret(UInt8, read(infil));
data

In [ ]:
# turn the data vector into a matrix of 12 values matching hexadecimal bytes - see DWTP 2.1 p.18
cols = 12
rows = Int(length(data) / cols)
mat = reshape(:, view(data), cols, :);

In [ ]:
data

In [ ]:
Data

In [ ]:
cols = 12
rows = Int(length(data) / cols)
reshape(view(data, :), :, cols)

In [ ]:
data = reinterpret(UInt8, read(infil));

# turn the data vector into a matrix of 12 values matching hexadecimal bytes - see DWTP 2.1 p.18
cols = 12
rows = Int(length(data) / cols)
mat = reshape(view(data, :), cols, :);

# Interleave last 4 matrix columns to form packet vector
## based on mschauer @ https://discourse.julialang.org/t/combining-two-arrays-with-alternating-elements/15498/2
packet = collect(Iterators.flatten(zip(mat[10,:], mat[11,:], mat[12,:])));

## get data for the Heave, North, and West displacements
Data3 = []
# Convert binary data to hexidecimal vectors
j = 0
println("Building displacements vectors - this takes a while!")
flush(stdout)
push!(Data3,[join([string(data[i], base = 16, pad = 2) for i = j+1 : j+12])[1:18] for j in 0:12:length(data)-12]);

In [ ]:
## get data for the Heave, North, and West displacements
@time begin

    Data = []

    # Convert binary data to hexidecimal vectors
    j = 0
    println("Building displacements vectors - this takes a while!")

    while true

        try
            aa = []

            for i = j*12+1 : j*12+12

                push!(aa, string(data[i], base = 16, pad = 2))

            end

            push!(Data, join(aa)[1:18])

        catch

            # escape if something is amiss        
            break

        end

        j = j + 1

    end
        
end


In [ ]:
@time begin

    ## get data for the Heave, North, and West displacements
    Data = []
    # Convert binary data to hexidecimal vectors
    j = 0
    println("Building displacements vectors - this takes a while!")
    flush(stdout)
    push!(Data,[join([string(data[i], base = 16, pad = 2) for i = j+1 : j+12])[1:18] for j in 0:12:length(data)-12]);
    
end

In [ ]:
collect(Iterators.flatten(zip(string(data[1:18,:], base = 16, pad = 2))))

In [ ]:
push!(Data,[join([string(data[i], base = 16, pad = 2) for i = j+1 : j+12])[1:18] for j in 0:12:length(data)-12])

In [ ]:
packet = collect(Iterators.flatten(zip(data[10:12,:])))